In [1]:
"""
Module to facilitate trading through Interactive Brokers's API
see: https://interactivebrokers.github.io/tws-api/index.html
Brent Maranzano
Dec. 14, 2018
Classes
    IBClient (EClient): Creates a socket to TWS or IBGateway, and handles
        sending commands to IB through the socket.
    IBWrapper (EWrapper): Hanldes the incoming data from IB. Many of these
        methods are callbacks from the request commands.
    IBApp (IBWrapper, IBClilent): This provides the main functionality. Many
        of the methods are over-rides of the IBWrapper commands to customize
        the functionality.
"""

import os.path
import time
import logging
import threading
import json
import numpy as np
import pandas as pd
from datetime import datetime
from ibapi import wrapper
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.common import OrderId, ListOfContractDescription, BarData,\
            HistogramDataList, TickerId, \
            TickAttrib, TickAttribBidAsk, TickAttribLast,HistoricalTick,HistoricalTickBidAsk,HistoricalTickLast,ListOfHistoricalTick,ListOfHistoricalTickBidAsk,ListOfHistoricalTickLast
from ibapi.order import Order
from ibapi.order_state import OrderState

API_THREAD = None



def setup_logger():
    """Setup the logger.
    """
    if not os.path.exists("log"):
        os.makedirs("log")

    time.strftime("pyibapi.%Y%m%d_%H%M%S.log")

    recfmt = "(%(threadName)s) %(asctime)s.%(msecs)03d %(levelname)s" \
             "%(filename)s:%(lineno)d %(message)s"

    timefmt = '%y%m%d_%H:%M:%S'

    logging.basicConfig(
        filename=time.strftime("log/pyibapi.%y%m%d_%H%M%S.log"),
        filemode="w",
        level=logging.INFO,
        format=recfmt, datefmt=timefmt
    )
    logger = logging.getLogger()
    console = logging.StreamHandler()
    console.setLevel(logging.ERROR)
    logger.addHandler(console)
    logging.debug("now is %s", datetime.now())


class IBClient(EClient):
    """Subclass EClient, which delivers message to the TWS API socket.
    """
    def __init__(self, app_wrapper):
        EClient.__init__(self, app_wrapper)


class IBWrapper(wrapper.EWrapper):
    """Subclass EWrapper, which translates messages from the TWS API socket
    to the program.
    """
    def __init__(self):
        wrapper.EWrapper.__init__(self)


class HistoricalRequestError(Exception):
    """Exceptions generated during requesting historical stock price data.
    """
    def __init__(self, message, errors):
        super().__init__(message)

        self.errors = errors
        self.message = message


class IBApp(IBWrapper, IBClient):
    """Main program class. The TWS calls nextValidId after connection, so
    the method is over-ridden to provide an entry point into the program.
    class variables:
    saved_contracts (dict): keys are symbols, values are dictionaries of
        information to uniquely define a contract used for stock trading.
        {symbol: {'contract_info_dictionary'}}
    saved_orders (dict): keys are order ids, values are Order, Contract
        {id: {order: Order, contract: Contract}}
    TODO
    positions
    """
    def __init__(self):
        IBWrapper.__init__(self)
        IBClient.__init__(self, app_wrapper=self)

        self.order_id = None
        self.saved_contract_details = {}
        self.positions = []
        self._contract_details = {}
        self._saved_orders = {}
        self._open_orders = []
        self._historical_data = []
        self._historical_data_req_end = False
        self._histogram = None
        self._load_contracts('contract_file.json')
        

    def error(self, reqId: TickerId, errorCode: int, errorString: str):
        """Overide EWrapper error method.
        """
        super().error(reqId, errorCode, errorString)
        print(reqId)

    def _load_contracts(self, filename):
        """Load saved contracts.
        """
        try:
            with open(filename, mode='r') as file_obj:
                self.saved_contracts = json.load(file_obj)
        except FileNotFoundError:
            pass

    def _save_contracts(self):
        """Save contracts.
        """
        with open("contracts.json", mode='a') as file_obj:
            json.dump(self._contract_details, file_obj)

    def nextValidId(self, orderId: int):
        """Method of EWrapper.
        Sets the order_id class variable.
        This method is called from after connection completion, so
        provides an entry point into the class.
        """
        super().nextValidId(orderId)
        self.order_id = orderId
        return self

    def _get_next_order_id(self):
        """Retrieve the current class variable order_id and increment
        it by one.
        Returns (int) current order_id
        """
        #  reqIds can be used to update the order_id, if tracking is lost.
        # self.reqIds(-1)
        current_order_id = self.order_id
        self.order_id += 1
        return current_order_id

    def get_contract_details(self, symbol=None):
        """Find the contract (STK, USD, NYSE|NASDAY.NMS|ARGA) for the symbol.
        Upon execution of IB backend, the EWrapper.symbolSamples is called,
        which is over-ridden to save the contracts to a class dictionary.
        This function then monitors the class dictionary until
        the symbol is found and then returns the contract.
        Arguments:
            symbol (string): Ticker.
        Returns: (Contract) Contract for the symbol.
        """
        # If the symbol has not already been saved, look it up.
        if symbol not in self.saved_contract_details:
            self._contract_details = None
            # The IB server will call symbolSamples upon completion.
            self.reqMatchingSymbols(1001, symbol)

            # Loop until the server has completed the request.
            while self._contract_details is None:
                time.sleep(0.2)

            # Select the proper contract
            for contract in self._contract_details:
                if contract.symbol == symbol and contract.currency == "USD"\
                        and contract.secType == "STK":
                    # NYSE stock
                    if contract.primaryExchange == "NYSE":
                        break
                    # Common ETFs
                    elif contract.primaryExchange == "ARCA":
                        break
                    # Nasdaq stock
                    elif contract.primaryExchange == "NASDAQ.NMS":
                        contract.primaryExchange = "ISLAND"
                        break

            # Save the contract information needed for defining a Contract.
            self.saved_contract_details[symbol] = {
                'currency': contract.currency,
                'secType': contract.secType,
                'exchange': "SMART",
                'primaryExchange': contract.primaryExchange,
                'secIdType': contract.secIdType,
                'secId': contract.secId
            }

        return self.saved_contract_details[symbol]

    def symbolSamples(self, reqId: int,
                      contractDescriptions: ListOfContractDescription):
        """Callback from reqMatchingSymbols. Add contracts that are of
        secType=STK, currency=USD, and primaryExchange=(NYSE | ISLAND) to the
        class variable contract_search_results.
        """
        super().symbolSamples(reqId, contractDescriptions)

        # Add all contracts to the to a list that the calling function can
        # access.
        contracts = []
        for desc in contractDescriptions:
            contracts.append(desc.contract)
        # is complete.
        self._contract_details = contracts

    def make_contract(self, symbol):
        """Create a contract for the given symbol.
        Arguments:
        symbol (str): Ticker symbol
        """
        contract_info = self.get_contract_details(symbol)
        contract = Contract()
        contract.symbol = symbol
        contract.currency = contract_info['currency']
        contract.exchange = contract_info['exchange']
        contract.primaryExchange = contract_info['primaryExchange']
        contract.secType = contract_info['secType']
        contract.secId = contract_info['secId']

        return contract

    def get_positions(self):
        """Get the account positions. If the class variable, positions, exists,
        return that value, else call the EClient method reqPositions, wait for
        a short time and then return the class variable positions.
        Returns (dict): Dictionary of the positions information.
        """
        self.positions = []
        self.reqPositions()
        time.sleep(1)

        return pd.DataFrame.from_dict(self.positions).set_index('account')

    def position(self, account: str, contract: Contract, position: float,
                 avgCost: float):
        super().position(account, contract, position, avgCost)
        self.positions.append({
            'account': account,
            'symbol': contract.symbol,
            'secType': contract.secType,
            'position': position,
            'cost': avgCost
        })

    def positionEnd(self):
        """Cancel the position subscription after a return.
        """
        super().positionEnd()
        self.cancelPositions()

    def create_bracket_orders(self, req_orders=None):
        """Create orders, but do not place.
        Arguments:
        req_orders (list): list of dictionaries - keys are:
            symbol (str): Equity ticker symbol.
            instruction (str): "BUY" | "SELL"
            price (float): Order set price.
            quantity (float): Order quantity.
            outside_rth (bool): outside regular trading hours
            tif (str): Time in force "DAY" | "GTC"
            profit_price (float): Price for profit taking
            stop_price (float): Price for stop loss
            parent_id (int): Id of parent trade.
        """
        # If only a single contract (dict) is passed convert it
        # to a list with a single item.
        if not isinstance(req_orders, list):
            req_orders = [req_orders]

        for req_order in req_orders:
            contract = self.make_contract(symbol=req_order['symbol'])

            # Create the parent order
            order_id = self._get_next_order_id()
            parent = Order()
            parent.orderId = order_id
            parent.action = req_order['instruction']
            parent.orderType = "LMT"
            parent.totalQuantity = req_order['quantity']
            parent.lmtPrice = req_order['price']
            parent.outsideRth = req_order['outside_rth']
            parent.tif = req_order['tif']
            parent.transmit = False
            self._saved_orders[order_id] = {
                "order": parent, "contract": contract
            }

            # Create the profit taker order
            if req_order['profit_price'] is not None:
                order_id = self._get_next_order_id()
                profit_taker = Order()
                profit_taker.orderId = order_id
                profit_taker.action = "SELL"\
                    if req_order['instruction'] == "BUY" else "BUY"
                profit_taker.orderType = "LMT"
                profit_taker.totalQuantity = req_order['quantity']
                profit_taker.lmtPrice = req_order['profit_price']
                profit_taker.parentId = parent.orderId
                profit_taker.transmit = False
                self._saved_orders[order_id] = {
                    "order": profit_taker, "contract": contract
                }

            # Create stop loss order
            if req_order['stop_price'] is not None:
                order_id = self._get_next_order_id()
                stop_loss = Order()
                stop_loss.orderId = order_id
                stop_loss.action = "SELL"\
                    if req_order['instruction'] == "BUY" else "BUY"
                stop_loss.orderType = "STP"
                stop_loss.auxPrice = req_order['stop_price']
                stop_loss.totalQuantity = req_order['quantity']
                stop_loss.parentId = parent.orderId
                stop_loss.transmit = False
                self._saved_orders[order_id] = {
                    "order": stop_loss, "contract": contract
                }

    def create_trailing_stop_orders(self, req_orders=None):
        """Create a trailing stop order.
        Arguments:
        req_orders (list): list of dictionaries - keys are:
            symbol (str): Equity ticker symbol.
            instruction (str): "BUY" | "SELL"
            quantity (float): Order quantity.
            trail_stop_price (float): Trailing stop price
            trail_amount (float): Trailing amount in dollars.
            limit_offset (float): Offset of limit price
                for sell - limit offset is greater than trailing amount
                for buy - limit offset is less than trailing amount
            outside_rth (bool): outside regular trading hours
            tif (str): Time in force "DAY" | "GTC"
            parent_id (int): Id of parent trade.
        """
        # If only a single contract (dict) is passed convert it
        # to a list with a single item.
        if not isinstance(req_orders, list):
            req_orders = [req_orders]

        for req_order in req_orders:
            contract = self.make_contract(symbol=req_order['symbol'])

            # Create the order
            order_id = self._get_next_order_id()
            order = Order()
            order.orderId = order_id
            order.action = req_order['instruction']
            order.orderType = "TRAIL LIMIT"
            order.totalQuantity = req_order['quantity']
            order.trailStopPrice = req_order['trail_stop_price']
            order.auxPrice = req_order['trail_amount']
            order.lmtPriceOffset = req_order['limit_offset']
            order.outsideRth = req_order['outside_rth']
            order.tif = req_order['tif']
            order.transmit = False
            # TODO parent_id
            self._saved_orders[order_id] = {
                "order": order, "contract": contract
            }

    def create_stop_limit_orders(self, req_orders=None):
        """Create a trailing stop order.
        Arguments:
        req_orders (list): list of dictionaries - keys are:
            symbol (str): Equity ticker symbol.
            instruction (str): "BUY" | "SELL"
            quantity (float): Order quantity.
            stop_price (float): stop price
            limit_price (float): limit price.
            outside_rth (bool): outside regular trading hours
            tif (str): Time in force "DAY" | "GTC"
            profit_price (float): Profit taking price.
        """
        # If only a single contract (dict) is passed convert it
        # to a list with a single item.
        if not isinstance(req_orders, list):
            req_orders = [req_orders]

        for req_order in req_orders:
            contract = self.make_contract(symbol=req_order['symbol'])

            # Create the order
            order_id = self._get_next_order_id()
            order = Order()
            order.orderId = order_id
            order.action = req_order['instruction']
            order.orderType = "STP LMT"
            order.totalQuantity = req_order['quantity']
            order.lmtPrice = req_order['limit_price']
            order.auxPrice = req_order['stop_price']
            order.outsideRth = req_order['outside_rth']
            order.tif = req_order['tif']
            order.transmit = False
            self._saved_orders[order_id] = {
                "order": order, "contract": contract
            }

            # Create the profit taker order
            if req_order['profit_price'] is not None:
                profit_taker_order_id = self._get_next_order_id()
                profit_taker = Order()
                profit_taker.orderId = profit_taker_order_id
                profit_taker.action = "SELL"\
                    if req_order['instruction'] == "BUY" else "BUY"
                profit_taker.orderType = "LMT"
                profit_taker.totalQuantity = req_order['quantity']
                profit_taker.lmtPrice = req_order['profit_price']
                profit_taker.parentId = order.orderId
                profit_taker.transmit = False
                self._saved_orders[profit_taker_order_id] = {
                    "order": profit_taker, "contract": contract
                }

    def create_pegged_orders(self, req_orders=None):
        """Create a pegged to bench mark order.
        Arguments:
        req_orders (list): list of dictionaries - keys are:
            symbol (str): Equity ticker symbol.
            instruction (str): "BUY" | "SELL"
            quantity (float): Order quantity.
            starting_price (float): Order starting price.
            outside_rth (bool): outside regular trading hours
            tif (str): Time in force "DAY" | "GTC"
            peg_change_amount (float): Change of price for the target
            ref_change_amount (float): Change of price of the reference
            ref_contract_id (int): Contract ID of the reference
                SPY: ConID: 756733, exchange: ARCA
                QQQ: ConID: 320227571, exchange: NASDAQ
            ref_exchange (str): Exchange of the reference
            ref_price (float): Start price of the reference
            ref_lower_price (float): Lower ref price allowed
            ref_upper_price (float): Upper ref price allowed
        """
        # If only a single contract (dict) is passed convert it
        # to a list with a single item.
        if not isinstance(req_orders, list):
            req_orders = [req_orders]

        for req_order in req_orders:
            contract = self.make_contract(symbol=req_order['symbol'])

            # Create the parent order
            order_id = self._get_next_order_id()
            order = Order()
            order.orderId = order_id
            order.orderType = "PEG BENCH"
            order.action = req_order['instruction']
            order.totalQuantity = req_order['quantity']
            order.startingPrice = req_order['starting_price']
            order.isPeggedChangeAmountDecrease = False
            order.peggedChangeAmount = req_order['peg_change_amount']
            order.referenceChangeAmount = req_order['ref_change_amount']
            order.referenceContractId = req_order['ref_contract_id']
            order.referenceExchange = req_order['ref_exchange']
            order.stockRefPrice = req_order['ref_price']
            order.stockRangeLower = req_order['ref_lower_price']
            order.stockRangeUpper = req_order['ref_upper_price']
            order.transmit = False
            self._saved_orders[order_id] = {
                "order": order, "contract": contract
            }

    def get_saved_orders(self, symbol=None):
        """Return saved orders for symbol. If symbol is None
        return all saved orders.
        Returns (dict) {order_id: {order: order, contract: contract}}
        """
        if symbol is None:
            return self._saved_orders

        orders = dict()
        for oid, order in self._saved_orders.items():
            if order['contract'].symbol == symbol:
                orders[oid] = order
        return orders

    def place_order(self, order_id=None):
        """Place a saved order. from a previously created saved order with
        order_id.
        Arguments:
        order_id (int): The order_id of a previously created order.
        """
        if order_id in self._saved_orders:
            self.placeOrder(order_id, self._saved_orders[order_id]['contract'],
                            self._saved_orders[order_id]['order'])
        del self._saved_orders[order_id]

    def place_all_orders(self):
        """Place all the saved orders.
        """
        order_ids = list(self._saved_orders.keys())
        for order_id in order_ids:
            self.place_order(order_id=order_id)

    def get_open_orders(self):
        """Call the IBApi.EClient reqOpenOrders. Open orders are returned via
        the callback openOrder.
        """
        self.reqOpenOrders()

    def openOrder(self, orderId: OrderId, contract: Contract, order: Order,
                  orderState: OrderState):
        """Callback from reqOpenOrders(). Method is over-ridden from the
        EWrapper class.
        """
        super().openOrder(orderId, contract, order, orderState)
        self._open_orders.append({
            'order_id': orderId,
            'contract': contract,
            'order': order
        })

    def get_quotes(self, symbols=None):
        """Get a quote for the symbol. Callsback to
        Warning: This may incur fees!
        Arguments:
        symbols (str|list): Equity ticker symbol or list of ticker symbols.
        Returns (Panda Series): Last trade price for the symbols.
        """
        # If only a single symbol is passed convert it
        # to a list with a single item.
        if isinstance(symbols, str):
            symbols = [symbols]

        # Get the bar data for each symbol
        quotes = pd.Series(index=symbols)
        for symbol in symbols:
            quote = self._req_historical_data(
                symbol,
                end_date="",
                duration="2 D",
                size="1 min",
                info="TRADES",
                rth=False
            )
            quotes[symbol] = float(quote.iloc[-1]['close_price'])

        return quotes

    def get_price_history(self, symbols=None, start_date=None, end_date=None,
                          bar_size="1 day", rth=False):
        """Get the price history for symbols.
        Arguments:
        symbols (str|list): Equity ticker symbol or list of ticker symbols.
        start_date (datetime): First date for data retrieval.
        end_date (datetime): Last data for data retrieval.
        bar_size (str): Bar size (e.g. "1 min", "1 day", "1 month")
            for valid strings see:
               http://interactivebrokers.github.io/tws-api/historical_bars.html
        rth (bool): True to only return data within regular trading hours.
        return (pandas.DataFrame): Price history data.
        """
        if end_date is None:
            end_date = datetime.today()

        # If only a single symbol is passed convert it
        # to a list with a single item.
        if isinstance(symbols, str):
            symbols = [symbols]

        # Estimate a duration string for the given date span.
        # TODO fix duration of seconds
        duration = end_date - start_date
        if duration.days >= 365:
            duration = "{} Y".format(int(duration.days/365))
        elif duration.days < 365 and duration.days > 1:
            duration = "{} D".format(np.busday_count(start_date.date(), end_date.date()))
        else:
            duration = "{} S".format(duration.seconds)
        # Get the bar data for each symbol
        bars = {}
        for symbol in symbols:
            try:
                bars[symbol] = self._req_historical_data(
                    symbol,
                    end_date=end_date.strftime("%Y%m%d %H:%M:%S"),
                    duration=duration,
                    size=bar_size,
                    info="TRADES",
                    rth=rth
                )
            except HistoricalRequestError as err:
                print(err.message)

        # Format the bars dictionary for conversion into DataFrame
        bars = {(outerKey, innerKey): values for outerKey, innerDict
                in bars.items() for innerKey, values in innerDict.items()}
        bars = pd.DataFrame(bars)

        # Reindex the bars using real time stamps.
        if (bar_size.find("secs") != -1 or bar_size.find("min") != -1 or
            bar_size.find("hour") != -1):
            index = [datetime.strptime(d, "%Y-%m-%d %H:%M:%S")
                     for d in bars.index]
        else:
            index = [datetime.strptime(d, "%Y-%m-%d") for d in bars.index]
        bars.index = index

        # Try to get rid of any missing data.
        bars.fillna(method="ffill", inplace=True)

        return bars

    
    def _req_historical_data(self, symbol, end_date="", duration="20 D",
                             size="1 day", info="TRADES", rth=False):
        """Get historical data using reqHistoricalData. Upon completion the
        server will callback historicalData, which is overridden.
        http://interactivebrokers.github.io/tws-api/historical_bars.html#hd_duration
        Arguments:
        symbol (str): Ticker symbol
        end_date (datetime): Last date requested
        duration (str): How far to go back - valid options: (S, D, W, M, Y)
        size (str): Bar size (see link)
        info (str): Type of data to return (see link)
        rth (bool): Return data only in regular trading hours"""
        contract = self.make_contract(symbol)

        self._historical_data = []
        self._historical_data_req_end = False
        self.reqHistoricalData(2001, contract, end_date, duration, size,
                               info, rth, 1, False, [])

        # Wait until the request has returned (make it blocking).
        start_time = datetime.now()
        while self._historical_data_req_end is not True:
            if (datetime.now() - start_time).microseconds > 1000000:
                raise HistoricalRequestError(
                    "Timeout occurred while retrieving price data for {}"
                    .format(symbol),
                    "_req_historical_data({})".format(symbol)
                )
            time.sleep(0.2)

        # Convert the data into
        bars_index = [b.date[:4]+"-"+b.date[4:6]+"-"+b.date[6:]
                      for b in self._historical_data]
        bars_data = [[float(b.open), float(b.high), float(b.low),
                      float(b.close), float(b.volume)]
                     for b in self._historical_data]

        bars = pd.DataFrame(
            index=bars_index,
            columns=['open_price', 'high', 'low', 'close_price', 'volume'],
            data=bars_data
        )

        return bars
    def historicalData(self, reqId: int, bar: BarData):
        """Overridden method from EWrapper. Checks to make sure reqId matches
        the self.historical_data[req_id] to confirm correct symbol.
        """
        self._historical_data.append(bar)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        """Overrides the EWrapper method.
        """
        self._historical_data_req_end = True

    def get_histogram(self, symbol=None, period="20 days"):
        """Get histograms of the symbols.
        Arguments:
        symbol (str): Equity ticker symbol or list of ticker symbols.
        period (str): Number of days to collect data.
        Returns (?): Histograms of the symbols
        """
        # If only a single symbol is passed convert it
        # to a list with a single item.

        contract = self.make_contract(symbol)
        self._histogram = None
        self.reqHistogramData(3001, contract, False, period)
        while self._histogram is None:
            time.sleep(0.2)

        histogram = pd.DataFrame(
            columns=["price", "count"],
            data=[[float(p.price), int(p.count)] for p in self._histogram]
        )

        return histogram

    def histogramData(self, reqId: int, items: HistogramDataList):
        """EWrapper method called from reqHistogramData.
        http://interactivebrokers.github.io/tws-api/histograms.html
        """
        self._histogram = items

    def keyboardInterrupt(self):
        """Stop exectution.
        """
        pass

    def quick_bracket(self, symbol=None, instruction=None, quantity=None,
                      amount=1000, limit_percent=None, profit_percent=None):
        """Calculate bracket order for symbol using a limit provided by
        limit_percent.
        Arguments
        symbol (str): Ticker symbol
        instruction (str): "BUY" | "SELL"
        quantity (int): Number of shares
        amount (float): Amount in dollars to trade
        limit_percent (float): Percent change from current quote to set limit.
        profit_percent (float): Percent change from limit price to take profit.
        Returns (dict) Parameters necessary to place a bracket order.
        """
        # Calculate a reasonable change if limit_percent is not given.
        if limit_percent is None:
            if instruction == "BUY":
                limit_percent = -0.3
            if instruction == "SELL":
                limit_percent = 0.3

        # Calculate a reasonable change if limit_percent is not given.
        if profit_percent is None:
            if instruction == "BUY":
                profit_percent = 0.3
            if instruction == "SELL":
                profit_percent = -0.3

        # Get the quote
        quote = self.get_quotes(symbol).loc[symbol]

        # Calculate the limit price from the limit_percent.
        limit_price = round(quote * (1 + limit_percent/100.), 2)
        # Calculate the profit price from the limit_price.
        profit_price = round(limit_price * (1 + profit_percent/100.), 2)

        # Calculate quantity if amount was provided.
        if quantity is None:
            quantity = int(amount / quote)

        req_order = {
            'symbol': symbol,
            'instruction': instruction,
            'quantity': quantity,
            'price': limit_price,
            'tif': "DAY",
            'outside_rth': True,
            'profit_price': profit_price,
            'stop_price': None
        }
        self.create_bracket_orders(req_orders=[req_order])

        for order_id in list(self.get_saved_orders(symbol).keys()):
            self.place_order(order_id=order_id)
    
    def request_ticks(self,symbol):
        contract = self.make_contract(symbol)
        self.reqHistoricalTicks(18001, contract,"20200828 21:39:33", "", 10, "TRADES", 1, True, [])
        print("got data")
    
    def historicalTicks(self, reqId: int,done: bool ,ticks = HistoricalTick):
         for tick in ticks:
             print("HistoricalTick. ReqId:", reqId, tick)
def main(port=7497):
    """Entry point into the program.
    Arguments:
    port (int): Port number that IBGateway, or TWS is listening.
    """
    global API_THREAD
    try:
        app = IBApp()
        app.connect("host.docker.internal", port, clientId=0)
        print("serverVersion:%s connectionTime:%s" % (app.serverVersion(),
                                                      app.twsConnectionTime()))
        API_THREAD = threading.Thread(target=app.run)
        API_THREAD.start()
        return app
    except KeyboardInterrupt:
        pass

if __name__ == "__main__":
    import sys
    # port number socker server is using (paper: 7497, live: 7496)
    # PORT_NUMBER = sys.argv[1]
    PORT_NUMBER = 7497
    app = main(port=PORT_NUMBER)



ruinnn
serverVersion:156 connectionTime:b'20200830 16:25:57 EST'


ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [2]:
app.request_ticks('TSLA')

got data


In [5]:
app.historicalTicks(18001,True,HistoricalTick)

NameError: name 'self' is not defined

In [5]:
a = app.get_price_history( symbols=['TSLA'], start_date=datetime(2020,8,5,12), end_date=datetime(2020,8,5,16),
                  bar_size="10 secs", rth=False)
display(a)
# os.chdir('C:\\S\\stocks')
try:a.to_csv("C:/S/a.csv")
except Exception as e: print(e)



TSLA                                     
                    open_price     high      low close_price volume
2020-08-05 12:00:00    1480.88  1480.88  1480.87     1480.88    1.0
2020-08-05 12:00:10    1480.88  1480.88  1480.87     1480.88    0.0
2020-08-05 12:00:20    1480.88  1480.88  1480.87     1480.88    0.0
2020-08-05 12:00:30    1480.36  1480.36  1479.61     1479.63    9.0
2020-08-05 12:00:40    1479.63  1479.63  1479.62     1479.63    0.0
...                        ...      ...      ...         ...    ...
2020-08-05 15:59:10    1485.78  1485.98  1484.99     1485.00   21.0
2020-08-05 15:59:20    1485.32  1485.46  1484.99     1485.13   12.0
2020-08-05 15:59:30    1485.22  1485.87  1485.22     1485.77   13.0
2020-08-05 15:59:40    1485.56  1486.48  1485.56     1486.02   34.0
2020-08-05 15:59:50    1485.60  1486.22  1485.10     1485.18   21.0

[1440 rows x 5 columns]

[Errno 2] No such file or directory: 'C:/S/a.csv'


In [5]:
### TEST NORMAL 
pd.set_option("display.max_rows", None, "display.max_columns", None)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.rcParams["figure.figsize"] = [15, 10]

b=a.copy()
b.columns=b.columns.droplevel()
# display(b)
def setMA(df,p_swma=10,p_lwma=20,p_signal=3,p_rsi=36):
    exp1 = df.close_price.ewm(span=p_swma, adjust=False).mean()
    exp2 = df.close_price.ewm(span=p_lwma, adjust=False).mean()
    df['close_ch'] = df.close_price.diff(1)
    df['swma'] = exp1
    df['lwma'] = exp2
    df['macd'] = exp1-exp2
    df['macd_ch'] = df.macd.diff(1)
    df['signal'] = b.macd.ewm(span=p_signal, adjust=False).mean()
    df['signal_ch'] = df.signal.diff(1)
    df['conv']=df.macd_ch-df.signal_ch
    df['conv_ch']=df.macd_ch-df.signal_ch
    
    close = df['close_price']
    delta = close.diff() 
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.ewm(com=p_rsi - 1, adjust=False).mean()
    roll_down = down.ewm(com=p_rsi - 1, adjust=False).mean().abs()
    rs = roll_up / roll_down   # relative strength =  average gain/average loss
    rsi = 100-(100/(1+rs))
    df['rsi'] = rsi
    
    return df[p_lwma-1:]

df = setMA(b)
# display(df)
fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02)
candlesticks = go.Candlestick(x=df.index,
                open=df['open_price'],
                high=df['high'],
                low=df['low'],
                close=df['close_price'])
swma = go.Scatter(x=df.index, y=df.swma, line=dict(color='orange', width=1))
lwma = go.Scatter(x=df.index, y=df.lwma, line=dict(color='green', width=1))

macd = go.Scatter(x=df.index, y=df.macd, line=dict(color='blue', width=1),yaxis="y2")
signal = go.Scatter(x=df.index, y=df.signal, line=dict(color='red', width=2),yaxis="y2")

rsi = go.Scatter(x=df.index, y=df.rsi, line=dict(color='gray', width=2),yaxis="y3")

data = [candlesticks,swma,lwma,macd,signal,rsi]

layout = go.Layout( yaxis=dict(
        domain=[.33, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[.2, .33]
    ),
    yaxis3=dict(
    domain=[0, .2]
    ),
)

fig = go.Figure(data=data,layout=layout)
fig.show()

stats = {
        'state':0,
    
        'open':0,
        'close':0,
        'profit':0,
        'pos':0,
    
        'signal':0,
        'signalp':0,
    
        'cross':-1
}
def momentumState(macd,signal,price,rsi):
    stats['signalp']=stats['signal']
    if signal>=macd:stats['signal']=1 
    else:stats['signal']=-1
    
    
    if stats['signal'] != stats['signalp']:
        
        stats['cross']= stats['cross']+1
        stats['state']= stats['signal']
        if stats['cross']<=0: stats['state']=0
        else: 
            stats['state'] = stats['signal']
            if stats['open'] == 0:
                print(f"OPEN {stats['signal']}:"+str(price))
                stats['open']=price
            else:
                stats['close'] = price
                stats['pos']+=1
                if stats['signalp']==1:
                    stats['profit']+=(stats['close']-stats['open'])
#                     print((stats['close']-stats['open']))
                    print("CLOSE :"+str(price)+f" | PROFIT: {(stats['close']-stats['open'])}")
                    stats['open'],stats['close'] = price,0
                    print(f"OPEN {stats['signal']}:"+str(price))
                if stats['signalp']==-1:
                    stats['profit']+=(stats['open']-stats['close'])
                    print("CLOSE :"+str(price)+f" | PROFIT: {(stats['open']-stats['close'])}")
#                     print((stats['open']-stats['close']))
                    stats['open'],stats['close'] = price,0
                    print(f"OPEN {stats['signal']}:"+str(price))
        
#     print(stats['signalp'],stats['signal'],stats['cross'])
    return stats['state']
df['state'] = df.apply(lambda x: momentumState(x.macd, x.signal,x.close_price,x.rsi),1)
print(stats['profit']*15,stats['pos'])
display(df)



ModuleNotFoundError: No module named 'plotly'

In [13]:
### TEST NORMAL 
pd.set_option("display.max_rows", None, "display.max_columns", None)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.rcParams["figure.figsize"] = [15, 10]

b=a.copy()
b.columns=b.columns.droplevel()

def setMA(df,p_swma=8,p_lwma=24,p_signal=6,p_rsi=36):
    exp1 = df.close_price.ewm(span=p_swma, adjust=False).mean()
    exp2 = df.close_price.ewm(span=p_lwma, adjust=False).mean()
    df['close_ch'] = df.close_price.diff(1)
    df['swma'] = exp1
    df['lwma'] = exp2
    df['macd'] = exp1-exp2
    df['macd_dir'] = ""
    df['macd_ch'] = df.macd.diff(1)
    df['signal'] = b.macd.ewm(span=p_signal, adjust=False).mean()
    df['signal_ch'] = df.signal.diff(2)/2
    df['conv']=df.macd_ch-df.signal_ch
    df['conv_ch']=df.macd_ch-df.signal_ch
    df['conv_ch_ewma']=df.conv_ch.ewm(span=4,adjust=False).mean()
    close = df['close_price']
    delta = close.diff() 
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.ewm(com=p_rsi - 1, adjust=False).mean()
    roll_down = down.ewm(com=p_rsi - 1, adjust=False).mean().abs()
    rs = roll_up / roll_down   # relative strength =  average gain/average loss
    rsi = 100-(100/(1+rs))
    df['rsi'] = rsi
    
    return df[p_lwma-1:]

df = setMA(b)

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02)
candlesticks = go.Candlestick(x=df.index,
                open=df['open_price'],
                high=df['high'],
                low=df['low'],
                close=df['close_price'])
swma = go.Scatter(x=df.index, y=df.swma, line=dict(color='orange', width=1))
lwma = go.Scatter(x=df.index, y=df.lwma, line=dict(color='green', width=1))

macd = go.Scatter(x=df.index, y=df.macd, line=dict(color='blue', width=1),yaxis="y2")
signal = go.Scatter(x=df.index, y=df.signal, line=dict(color='red', width=2),yaxis="y2")

rsi = go.Scatter(x=df.index, y=df.rsi, line=dict(color='gray', width=2),yaxis="y3")

data = [candlesticks,swma,lwma,macd,signal,rsi]

layout = go.Layout( yaxis=dict(
        domain=[.33, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[.2, .33]
    ),
    yaxis3=dict(
    domain=[0, .2]
    ),
)

fig = go.Figure(data=data,layout=layout)
fig.show()

stats = {
        'state':0,
    
        'open':0,
        'close':0,
        'profit':0,
        'pos':0,
    
        'signal':0,
        'signalp':0,
    
        'cross':-1
}
def momentumState(macd,signal,signal_ch,price,rsi):
    stats['signalp']=stats['signal']
    if signal>=macd:stats['signal']=-1 
    elif signal<macd:stats['signal']=1
    
    
    if stats['signal'] != stats['signalp']:
        stats['cross']= stats['cross']+1
        
    if stats['cross']<=0: stats['state']=0
    else: 
        if stats['state']==0 and stats['signal']==1 and signal>=0:
            stats['state'] = stats['signal']
            stats['open'] = price
        if stats['state'] == 1 and signal_ch <= 0:
            stats['close'] = price
            stats['state'] = 0
            print(stats['close']-stats['open'])
            stats['open'] = 0
        elif stats['signal']==-1 and signal <0:
            stats['state'] = stats['signal']
            stats['open'] = price
    return [stats['state'],"test"]
df['state'] = df.apply(lambda x: momentumState(x.macd, x.signal,x.signal_ch,x.close_price,x.rsi),1)
print(stats['profit']*15,stats['pos'])
display(df)



C:\Users\mysli\AppData\Local\Continuum\anaconda3\envs\webauto\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0 0


,open_price,high,low,close_price,volume,close_ch,swma,lwma,macd,macd_dir,macd_ch,signal,signal_ch,conv,conv_ch,conv_ch_ewma,rsi,state
2020-08-03 12:03:50,1498.93,1498.94,1498.93,1498.93,1.0,0.14,1498.308035,1498.323875,-0.015840,,0.124998,-0.255717,0.090142,0.034856,0.034856,0.067323,86.364169,"[0, test]"
2020-08-03 12:04:00,1499.05,1499.05,1498.25,1498.25,8.0,-0.68,1498.295138,1498.317965,-0.022827,,-0.006987,-0.189177,0.081246,-0.088232,-0.088232,0.005101,84.675248,"[0, test]"
2020-08-03 12:04:10,1497.97,1497.98,1497.39,1497.39,5.0,-0.86,1498.093997,1498.243728,-0.149731,,-0.126905,-0.177907,0.038905,-0.165810,-0.165810,-0.063263,82.574633,"[0, test]"
2020-08-03 12:04:20,1497.28,1497.48,1496.94,1497.48,5.0,0.09,1497.957553,1498.182630,-0.225077,,-0.075345,-0.191384,-0.001103,-0.074242,-0.074242,-0.067655,82.621041,"[-1, test]"
2020-08-03 12:04:30,1497.55,1497.55,1496.05,1497.17,39.0,-0.31,1497.782541,1498.101619,-0.319078,,-0.094001,-0.227868,-0.024981,-0.069021,-0.069021,-0.068201,81.848759,"[-1, test]"
2020-08-03 12:04:40,1496.71,1497.02,1496.70,1497.02,2.0,-0.15,1497.613088,1498.015090,-0.402002,,-0.082924,-0.277621,-0.043118,-0.039806,-0.039806,-0.056843,81.469753,"[-1, test]"
2020-08-03 12:04:50,1497.63,1497.63,1495.43,1496.27,38.0,-0.75,1497.314624,1497.875482,-0.560859,,-0.158857,-0.358546,-0.065339,-0.093518,-0.093518,-0.071513,79.574735,"[-1, test]"
2020-08-03 12:05:00,1496.55,1496.55,1494.50,1494.50,6.0,-1.77,1496.689152,1497.605444,-0.916292,,-0.355433,-0.517902,-0.120141,-0.235293,-0.235293,-0.137025,75.321843,"[-1, test]"
2020-08-03 12:05:10,1494.00,1495.01,1494.00,1494.35,18.0,-0.15,1496.169340,1497.345008,-1.175668,,-0.259376,-0.705835,-0.173645,-0.085731,-0.085731,-0.116507,74.972571,"[-1, test]"
2020-08-03 12:05:20,1495.22,1495.23,1494.74,1495.16,24.0,0.81,1495.945042,1497.170208,-1.225165,,-0.049497,-0.854215,-0.168157,0.118659,0.118659,-0.022441,75.600982,"[-1, test]"
